In [4]:
!pip install PyPDF2

## Librerias

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

## Cargar datos desde PDF

In [6]:
import PyPDF2
import os

def cargar_datos_desde_pdf(ruta_pdf):
    """
    Cargar datos desde un archivo PDF y extraer su texto.
    :param ruta_pdf: Ruta del archivo PDF
    :return: Texto extraído del PDF
    """
    texto_completo = ""
    try:
        with open(ruta_pdf, 'rb') as archivo:
            lector_pdf = PyPDF2.PdfReader(archivo)
            for pagina in lector_pdf.pages:
                texto_pagina = pagina.extract_text()
                if texto_pagina:
                    texto_completo += texto_pagina
        return texto_completo
    except FileNotFoundError:
        print("Error: El archivo PDF no se encuentra. Verifica la ruta.")
    except Exception as e:
        print(f"Error al leer el PDF: {e}")
    return None

# Verificar si el archivo existe
ruta_pdf = './Historia.pdf'

if os.path.isfile(ruta_pdf):
    contenido_pdf = cargar_datos_desde_pdf(ruta_pdf)
else:
    print("Error: No se encuentra el archivo PDF.")


## Seleccionamos el Modelo que vamos a utilizar

In [7]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/home/bigdata/miniconda3/envs/proyecto-bida/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Accedemos a la base de datos

In [8]:
vector_store = Chroma(
    collection_name="rag2",
    embedding_function=embeddings,
    persist_directory="./datasets",
)

In [9]:
# Se ve como funciona la base de datos
retriever = vector_store.as_retriever()

## RAG Chain

In [10]:
# Plantilla de conversación
conversation_template = """Por favor, responde la siguiente pregunta basándote solo en el contexto dado:
{context}

Pregunta: {question}
"""

# Crear el prompt a partir de la plantilla
prompt_template = ChatPromptTemplate.from_template(conversation_template)

# LLM local
local_llm = "tinyllama"
local_model = ChatOllama(model=local_llm)

# Cadena de procesamiento
pregunta = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt_template
    | local_model
    | StrOutputParser()
)

In [11]:
pregunta.invoke("¿Que es un PDF?")

'Sí, un PDF es una aplicación de archivo de imagen que contiene información fijada en páginas o documentos electrónicos. Además, las tablas, gráficos y otros recursos adjuntos también están incluidos dentro del formato PDF. Tanto la mayoría de las computadoras como los dispositivos móviles han incorporado una interfaz estándar de carga y descarga de documentos, así como otras herramientas para la edición, la impresión y la lectura de PDF.'